In [ ]:
import cv2
import numpy as np
import time

# 打开摄像头
video = cv2.VideoCapture(5)

# 设置相机分辨率为800x600
video.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
video.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# 设置相机帧率为60Hz
video.set(cv2.CAP_PROP_FPS, 60)

# 初始化计数器和计时器
frame_count = 0
start_time = time.time()

while True:
    # 从摄像头读取一帧
    ret, img = video.read()

    # 如果读取失败，退出循环
    if not ret:
        break

    # 将图像从BGR转换为HSV颜色空间
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # 定义蓝色的HSV范围
    lower_blue = np.array([100, 50, 50])
    upper_blue = np.array([130, 255, 255])

    # 创建蓝色的掩码
    blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # 对掩码应用高斯模糊
    blue_mask = cv2.GaussianBlur(blue_mask, (5, 5), 0)

    # 获取二值图像的尺寸
    whole_h, _ = blue_mask.shape[:2]

    # 在蓝色掩码中查找轮廓
    _,contours, _ = cv2.findContours(image=blue_mask, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)

    # 将轮廓转换为列表并按面积降序排列
    contours = sorted(contours, key=lambda c: cv2.contourArea(c), reverse=True)[:5]

    # 存储矩形宽度、高度和点的数组
    rects = []

    # 遍历前5个轮廓
    for cont in contours:
        # 获取边界矩形的坐标和尺寸
        x, y, w, h = cv2.boundingRect(cont)
        
        # 检查有效矩形的条件
        if h / w >= 2 and h / whole_h > 0.1 and h > w:
            rects.append([x, y, w, h])

    # 找到面积最接近的两个矩形
    min_value = float('inf')
    best_rects = []

    for i in range(len(rects) - 1):
        for j in range(i + 1, len(rects)):
            value = abs(rects[i][2] * rects[i][3] - rects[j][2] * rects[j][3])
            if value < min_value:
                min_value = value
                best_rects = [rects[i], rects[j]]

    # 如果找到了两个最近矩形
    if best_rects:
        # 计算并打印矩形的四个角点
        rectangle1, rectangle2 = best_rects
        point1 = [rectangle1[0] + rectangle1[2] / 2, rectangle1[1]]
        point2 = [rectangle1[0] + rectangle1[2] / 2, rectangle1[1] + rectangle1[3]]
        point3 = [rectangle2[0] + rectangle2[2] / 2, rectangle2[1]]
        point4 = [rectangle2[0] + rectangle2[2] / 2, rectangle2[1] + rectangle2[3]]

        print(point1, point2, point3, point4)

        # 定义矩形为多边形并在原始图像上绘制它
        pts = np.array([point1, point2, point4, point3], np.int32).reshape((-1, 1, 2))
        cv2.polylines(img, [pts], isClosed=True, color=(0, 255, 0), thickness=2)

    # 显示摄像头画面
    cv2.imshow('Camera Feed', img)

    # 计数器递增
    frame_count += 1

    # 计时器结束时计算帧率
    if time.time() - start_time >= 1.0:
        fps = frame_count / (time.time() - start_time)
        print(f"FPS: {fps:.2f}")
        print(f"FPS: 88")

        # 重置计数器和计时器
        frame_count = 0
        start_time = time.time()

    # 检查是否按下 'q' 键以退出循环
    if cv2.waitKey(1) & 0xFF == ord('q'):
        breakq

# 释放摄像头资源和关闭窗口
video.release()
cv2.destroyAllWindows()
